## **Asymmetry**

In [119]:
import pandas as pd
import statsmodels.api as sm
from matplotlib import pyplot as plt
import numpy as np

# read data from excel file 
df = pd.read_excel('ET_ToDoModerations.xlsx', sheet_name='Tabelle1')

# convert columns to numeric
columns_to_convert = ['MeanEntO']
df[columns_to_convert] = df[columns_to_convert].apply(pd.to_numeric, errors='coerce')

# choose target columns and set index to subject
df_filtered = df[['Subject','MeanEntO']]
df_filtered.set_index('Subject', inplace=True)

# -----------------------------------------------------------------------------------------

# Load data from Excel files
data = pd.read_excel('CRB_Power_B1B2.xlsx')

# Define electrode pairs for calculation
electrode_pairs = [
    ('Fp2', 'Fp1'), ('F4', 'F3'), ('F8', 'F7'), ('FT10', 'FT9'), ('FC2', 'FC1'),
    ('FC6', 'FC5'), ('C4', 'C3'), ('T8', 'T7'), ('TP10', 'TP9'), ('CP2', 'CP1'),
    ('CP6', 'CP5'), ('P4', 'P3'), ('P8', 'P7'), ('O2', 'O1')
]

# Function to calculate the differences for each pair and condition
def calculate_differences(data, pairs, suffix):
    results = pd.DataFrame()
    for pair in pairs:
        diff_col_name = pair[0] + pair[1] + suffix  # e.g., Fp2Fp1_B1
        results[diff_col_name] = data[pair[0] + suffix] - data[pair[1] + suffix]
    return results

# Calculate differences for B1, B2, and B1B2 conditions
new_data = pd.DataFrame()
for suffix in ['_B1', '_B2', '_B1B2']:
    diffs = calculate_differences(data, electrode_pairs, suffix)
    new_data = pd.concat([new_data, diffs], axis=1)

# Add 'Subject', 'Pwelch', and 'Freq_range' columns from the original DataFrame
new_data['Subject'] = data['subid']
new_data['Pwelch'] = data['Freq_Names']
new_data['Freq_range'] = data['Freq_Range']

# Ensure the new DataFrame has the correct column order
cols = ['Subject', 'Pwelch', 'Freq_range'] + [col for col in new_data if col not in ['Subject', 'Pwelch', 'Freq_range']]
new_data_corrected = new_data[cols]

# Display the first few rows of the new DataFrame
new_data_corrected.head()


,Subject,Pwelch,Freq_range,Fp2Fp1_B1,F4F3_B1,F8F7_B1,FT10FT9_B1,FC2FC1_B1,FC6FC5_B1,C4C3_B1,...,FC2FC1_B1B2,FC6FC5_B1B2,C4C3_B1B2,T8T7_B1B2,TP10TP9_B1B2,CP2CP1_B1B2,CP6CP5_B1B2,P4P3_B1B2,P8P7_B1B2,O2O1_B1B2
0,1,Alpha1,8-13,0.385810,0.443402,0.439939,-0.614480,0.124590,0.293298,0.383456,...,0.136076,0.220748,0.248142,0.175325,-0.050729,-0.051844,-0.057642,-0.042727,-0.228514,1.230284
1,1,Alpha2,7.5-12.5,0.432096,0.480357,0.377144,-0.425650,0.144446,0.332155,0.489876,...,0.156844,0.282109,0.333860,0.038117,0.049941,-0.060415,0.013004,0.033637,-0.130899,1.474093
2,1,Alpha3,7.5-10,0.625690,0.135172,-1.003521,-1.503887,0.075778,0.132816,0.498405,...,0.074695,0.059381,0.372077,-0.000401,-0.267669,0.032630,0.074106,0.194987,-0.126508,0.102870
3,1,Alpha4,10-12.5,0.035523,0.740321,0.280599,-0.449304,0.217385,0.571834,0.768361,...,0.243683,0.538678,0.567926,-0.070628,0.371112,-0.108718,0.027038,-0.022079,0.304905,2.474690
4,1,Alpha5,7-13,-0.034177,0.118819,-1.107239,-0.628396,0.069245,0.151472,0.325979,...,0.075257,0.072267,0.200636,0.214444,0.063058,-0.059101,-0.002311,-0.047511,-0.284040,0.983036


In [96]:
# Function to generate electrode list for each condition
def generate_electrode_list(electrode_pairs, suffix):
    return [f"{pair[0]}{pair[1]}_{suffix}" for pair in electrode_pairs]

# Generate the lists for each condition
B1_electrodes = generate_electrode_list(electrode_pairs, 'B1')
B2_electrodes = generate_electrode_list(electrode_pairs, 'B2')
B1B2_electrodes = generate_electrode_list(electrode_pairs, 'B1B2')

B1B2_electrodes

['Fp2Fp1_B1B2',
 'F4F3_B1B2',
 'F8F7_B1B2',
 'FT10FT9_B1B2',
 'FC2FC1_B1B2',
 'FC6FC5_B1B2',
 'C4C3_B1B2',
 'T8T7_B1B2',
 'TP10TP9_B1B2',
 'CP2CP1_B1B2',
 'CP6CP5_B1B2',
 'P4P3_B1B2',
 'P8P7_B1B2',
 'O2O1_B1B2']

In [139]:
# Function to filter, merge, and re-index DataFrames based on condition
def process_condition(data, condition_electrodes, df_filtered, pwelch_condition='Beta'):
    # Filter data for the specified 'pwelch_condition'
    pwelch_data = data[data['Pwelch'] == pwelch_condition]
    
    # Create a list of DataFrames for each electrode in the condition
    df_pwelch = [df_filtered.merge(pwelch_data[['Subject', electrode]], on='Subject') for electrode in condition_electrodes]
    
    # Set the index to 'Subject' for each DataFrame in the list
    df_pwelch_with_index = [df.set_index('Subject') for df in df_pwelch]
    
    return df_pwelch_with_index

# Process each condition (B1, B2, B1B2)
df_pwelch_B1 = process_condition(new_data_corrected, B1_electrodes, df_filtered)
df_pwelch_B2 = process_condition(new_data_corrected, B2_electrodes, df_filtered)
df_pwelch_B1B2 = process_condition(new_data_corrected, B1B2_electrodes, df_filtered)

# Example of how to access and display the first few rows of the second DataFrame for B1 condition
df_pwelch_B1B2[1].head(11)

,MeanEntO,F4F3_B1B2
Subject,,
1,3.857143,0.113436
3,4.000000,0.089452
4,6.142857,0.242654
5,4.000000,0.082126
8,4.000000,0.010251
12,2.285714,0.015592
15,4.428571,1.603631
16,5.000000,-0.209990
17,4.714286,-0.004277


In [140]:
prob_f_statistic_list = list()

for i in range(len(B1B2_electrodes)):
    y = df_pwelch_B1B2[i]['MeanEntO']
    X = df_pwelch_B1B2[i][B1B2_electrodes[i]]
    X = sm.add_constant(X) # adding a constant

    # fit the model
    model = sm.OLS(y, X)
    results = model.fit()
    
    prob_f_statistic_list.append(results.f_pvalue)

prob_f_statistic_list

[0.8126757113928962,
 0.522361298712764,
 0.7481972455143406,
 0.9791550873950203,
 0.31056858459102576,
 0.164693990966096,
 0.8006886369237266,
 0.04357302531780002,
 0.37480540017871633,
 0.8062865608236371,
 0.4999408658694695,
 0.45528467925906113,
 0.8993741399046052,
 0.8870907833893475]